<a href="https://colab.research.google.com/github/Khang-CS/Khang-CS.github.io/blob/main/cbsdata_detection_v1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Custom Training with YOLOv5

In this tutorial, we assemble a dataset and train a custom YOLOv5 model to recognize the objects in our dataset. To do so we will take the following steps:

* Gather a dataset of images and label our dataset
* Export our dataset to YOLOv5
* Train YOLOv5 to recognize the objects in our dataset
* Evaluate our YOLOv5 model's performance
* Run test inference to view our model at work



![](https://uploads-ssl.webflow.com/5f6bc60e665f54545a1e52a5/615627e5824c9c6195abfda9_computer-vision-cycle.png)

# Step 1: Install Requirements

In [ ]:
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

isTrained = False # If the model is trained, set isTrained = True

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 14375, done.
remote: Counting objects: 100% (181/181), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 14375 (delta 114), reused 122 (delta 69), pack-reused 14194
Receiving objects: 100% (14375/14375), 13.37 MiB | 18.25 MiB/s, done.
Resolving deltas: 100% (9926/9926), done.
/content/yolov5
     |████████████████████████████████| 1.6 MB 47.1 MB/s 
     |████████████████████████████████| 54 kB 2.1 MB/s 
     |████████████████████████████████| 1.1 MB 61.5 MB/s 
     |████████████████████████████████| 138 kB 64.9 MB/s 
     |████████████████████████████████| 178 kB 73.8 MB/s 
     |████████████████████████████████| 145 kB 74.1 MB/s 
     |████████████████████████████████| 67 kB 6.8 MB/s 
     |████████████████████████████████| 62 kB 1.7 MB/s 
Setup complete. Using torch 1.12.1+cu113 (Tesla T4)


In [ ]:
%pip install clearml
#%pip install comet_ml
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=NQYHIB03J1MJ8ZYIE9XU
%env CLEARML_API_SECRET_KEY=PpWBOthD8r9UKIqPtPFahuUqmIiWIZJ97wGADilzn6obqgDDLF

# Step 2: Assemble Our Dataset

In order to train our custom model, we need to assemble a dataset of representative images with bounding box annotations around the objects that we want to detect. And we need our dataset to be in YOLOv5 format.

In Roboflow, you can choose between two paths:

* Convert an existing dataset to YOLOv5 format. Roboflow supports over [30 formats object detection formats](https://roboflow.com/formats) for conversion.
* Upload raw images and annotate them in Roboflow with [Roboflow Annotate](https://docs.roboflow.com/annotate).

In [ ]:
# set up environment
os.environ["DATASET_DIRECTORY"] = "/content/datasets"
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from roboflow import Roboflow
#rf = Roboflow(model_format="yolov5", notebook="ultralytics")

In [ ]:
#rf = Roboflow(api_key="9HL2xrImKT3ISNGWT2kp")
#from roboflow import Roboflow
rf = Roboflow(api_key="0isAJ601MTJaUzOMeLTw")
#project = rf.workspace("hpclab-cj0fx").project("cbscountthings")
#dataset = project.version(1).download("yolov5")
project = rf.workspace("hpclab-cj0fx").project("cbscountthings")
#dataset = project.version(4).download("yolov5")
#dataset_path = 

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to /content/datasets/cbscountthings-4 in yolov5pytorch:: 100%|██████████| 3452/3452 [00:22<00:00, 153.11it/s]


In [ ]:
isTrained = False

# Step 3: Train Our Custom YOLOv5 model

Here, we are able to pass a number of arguments:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** Our dataset locaiton is saved in the `dataset.location`
- **weights:** specify a path to weights to start transfer learning from. Here we choose the generic COCO pretrained checkpoint.
- **cache:** cache images for faster training

In [ ]:
!python train.py --img 640 --batch 16 --epochs 300 --data {dataset_path}/data.yaml --weights yolov5m6.pt --cache ram  

isTrained = True

train: weights=yolov5m6.pt, cfg=, data=/content/datasets/cbscountthings-4/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/ultralytics/yolov5
   1a93f9c..5f7b5d2  exp8       -> origin/exp8
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-205-geef9057 Python-3.7.15 torc

## Transfer learning: Retrain the model

In [ ]:
lastparam = "/content/drive/MyDrive/params/exp{}/best.pt".format(1)

In [ ]:
%pip install clearml
%pip install comet_ml
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
#%load_ext tensorboard
#%tensorboard --logdir runs

In [ ]:
#!python train.py --img 416 --batch 16 --epochs 2 --data {dataset.location}/data.yaml --weights $lastparam --cache ram
!python train.py --img 832 --batch-size -1 --epochs 2000 --data {dataset_path}/data.yaml  --weights {lastparam} --cache ram --project "/content/drive/MyDrive/Runs" --name Run_22Oct22  # --cache
#!python train.py --resume "/content/drive/MyDrive/Runs/Run_22Oct228/weights/last.pt"
isTrained = True

train: weights=/content/drive/MyDrive/params/exp1/best.pt, cfg=, data=/content/drive/MyDrive/CBSYOLODATA/cbscountthings-7/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=200, batch_size=8, imgsz=832, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=/content/drive/MyDrive/Runs, name=Run_22Oct22, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-205-geef9057 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, c

# Evaluate Custom YOLOv5 Detector Performance
Training losses and performance metrics are saved to Tensorboard and also to a logfile.

If you are new to these metrics, the one you want to focus on is `mAP_0.5` - learn more about mean average precision [here](https://blog.roboflow.com/mean-average-precision/).

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
# %reload_ext tensorboard
%load_ext tensorboard
%tensorboard --logdir runs

<IPython.core.display.Javascript object>

#Run Inference  With Trained Weights
Run inference with a pretrained checkpoint on contents of `test/images` folder downloaded from Roboflow.

In [ ]:
!gdown 1W5ADdLQIm0wf-1qmVrwwxT9tFSReMByG

Downloading...
From: https://drive.google.com/uc?id=1W5ADdLQIm0wf-1qmVrwwxT9tFSReMByG
To: /content/yolov5/best.pt
100% 42.1M/42.1M [00:00<00:00, 92.4MB/s]


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#bestparam = "/content/yolov5/best.pt  "
bestparam = "/content/yolov5/yolov5/best.pt  "
if isTrained:
  bestparam = "runs/train/exp/weights/best.pt" # param from training

In [ ]:
!python detect.py -h

In [ ]:
!python detect.py --weights $bestparam --img 832 --line-thickness 1 --hide-labels --conf 0.5 --source {dataset.location}/test/images

In [ ]:
import glob
from IPython.display import Image, display

In [ ]:
# display inference on ALL test images
for imageName in glob.glob('/content/yolov5/runs/detect/exp/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

In [ ]:
!gdown 1Pr5E3SdMLbAFNBl3MoxvQSqLZQ3RwNpY

In [ ]:
!unzip "/content/yolov5/new_sample.zip" -d "/content/sample_data"

In [ ]:
confs = [0.5, 0.4, 0.3]
for conf in confs:
    !python detect.py --weights $bestparam --img 800 --line-thickness 1 --conf $conf --source "/content/datasets/cbscountthings-4/test/images"

In [ ]:
for imageName in glob.glob('runs/detect/exp7/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

In [ ]:
for imageName in glob.glob('runs/detect/exp8/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

In [ ]:
for imageName in glob.glob('runs/detect/exp9/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

In [ ]:
#for imageName in glob.glob('runs/detect/exp9/*.jpg'): #assuming JPG
    #display(Image(filename=imageName))
    #print("\n")

# Conclusion and Next Steps

Congratulations! You've trained a custom YOLOv5 model to recognize your custom objects.

To improve you model's performance, we recommend first interating on your datasets coverage and quality. See this guide for [model performance improvement](https://github.com/ultralytics/yolov5/wiki/Tips-for-Best-Training-Results).

To deploy your model to an application, see this guide on [exporting your model to deployment destinations](https://github.com/ultralytics/yolov5/issues/251).

Once your model is in production, you will want to continually iterate and improve on your dataset and model via [active learning](https://blog.roboflow.com/what-is-active-learning/).

In [ ]:
#export your model's weights for future use
from google.colab import files
files.download('runs/train/exp/weights/best.pt')